# Installing the Dependencies

In [1]:
!pip install requests beautifulsoup4 selenium pandas flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 33.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [86]:
# Scraping the news_headlines


news_headlines=[]
base_url= 'https://timesofindia.indiatimes.com/topic/medical/news'

# To fetch the  all the pages
num_pages=9;



url=f"{base_url}"
html_text=requests.get(url)
if html_text.status_code != 200:
      print(f"Failed to fetch page. Status code: {html_text.status_code}")
      exit()
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_text.content, "html.parser")

# Find the news headlines and links

for article in soup.find_all('div',class_="uwU81"):
  link =article.find("a")["href"]
  if link.startswith("/"):
    link="https://timesofindia.indiatimes.com"+link
  headline = article.find('div', class_='fHv_i o58kM').get_text(strip=True)
  published_date = article.find('div', class_='ZxBIG').get_text(strip=True)
  news_headlines.append({"headline":headline,"link":link,"published_date":published_date})

In [87]:
news_headlines

[{'headline': '‘Improvemedicalfacilities in court’',
  'link': 'https://timesofindia.indiatimes.com/city/ranchi/improve-medical-facilities-in-court/articleshow/118427773.cms',
  'published_date': 'Feb 20, 2025, 23:38 (IST)'},
 {'headline': 'Students atmedicalcollege suspended for sexual harassment',
  'link': 'https://timesofindia.indiatimes.com/city/jaipur/students-at-medical-college-suspended-for-sexual-harassment/articleshow/118487301.cms',
  'published_date': 'TNN /Feb 22, 2025, 23:44 (IST)'},
 {'headline': 'medicalboard set up for abandoned baby',
  'link': 'https://timesofindia.indiatimes.com/city/thiruvananthapuram/medical-board-set-up-for-abandoned-baby/articleshow/118482906.cms',
  'published_date': 'TNN /Feb 22, 2025, 20:02 (IST)'},
 {'headline': 'PGmedical: Counselling schedule announced',
  'link': 'https://timesofindia.indiatimes.com/city/mumbai/pg-medical-counselling-schedule-announced/articleshow/118256856.cms',
  'published_date': 'Feb 14, 2025, 23:47 (IST)'},
 {'headli

In [82]:
len(news_headlines)

30

In [88]:
import requests
from bs4 import BeautifulSoup
def get_article_text(url):

    response=requests.get(url)

    if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            content = soup.find("div", class_="_s30J clearfix")
            article_text = content.get_text(separator="\n").strip()
            return article_text
    else:
        print(f"Failed to fetch article. Status code: {response.status_code}")
        return

In [89]:
for item in news_headlines:

    try:
        link = item['link']

        article_text = get_article_text(link)

        if article_text:
            item['article_text'] = article_text
        else:
            news_headlines.remove(item)
            print(f"No article text found for: {link}")
    except Exception as e:
        # remove the dict
        news_headlines.remove(item)
        print(f"Error processing {link}: {e}")

# Summarizing the news article

In [90]:
from transformers import pipeline

summarizer = pipeline("summarization")
for item in news_headlines:
  try:
    article=item['article_text']
    summary = summarizer(article, max_length=50, min_length=25, do_sample=False,truncation=True)
    item['summary']=summary

  except Exception as e:
    # remove the dict
    news_headlines.remove(item)
    print(f"Error processing {link}: {e}")



No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [91]:
news_headlines

[{'headline': '‘Improvemedicalfacilities in court’',
  'link': 'https://timesofindia.indiatimes.com/city/ranchi/improve-medical-facilities-in-court/articleshow/118427773.cms',
  'published_date': 'Feb 20, 2025, 23:38 (IST)',
  'article_text': 'Ranchi: The \nRanchi District Bar Association\n (RDBA) general secretary Sanjay Virdrohi and the association members have demanded improved \nmedical facilities\n at the \nRanchi civil court\n in order to tackle emergency cases.\nThe demand was voiced following the demise of a Jharkhand high court advocate Suraj Kumar on Tuesday due to heart attack on the court premises and the death of a civil court lawyer S N Tiwari on Wednesday in a similar manner during legal work outside the campus.\nVidrohi said, "There are two clinics on the civil court premises but they are mostly meant for primary medical facility. In case of emergency crucial time would be lost to take the patient to the nearby district hospital due to chaotic conditions of car parking 

# Keyword Extraction

In [92]:
import spacy

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

# Function to extract nouns and proper nouns
def extract_nouns(text):
    doc = nlp(text)
    return [token.text for token in doc if token.pos_ in ["NOUN", "PROPN"]]


In [93]:
for item in news_headlines:
  item['keywords']=extract_nouns(item['article_text'])
  print(item['keywords'])

['Ranchi', 'Ranchi', 'District', 'Bar', 'Association', 'RDBA', 'secretary', 'Sanjay', 'Virdrohi', 'association', 'members', 'facilities', 'Ranchi', 'court', 'order', 'emergency', 'cases', 'demand', 'demise', 'Jharkhand', 'court', 'advocate', 'Suraj', 'Kumar', 'Tuesday', 'heart', 'attack', 'court', 'premises', 'death', 'court', 'lawyer', 'S', 'N', 'Tiwari', 'Wednesday', 'manner', 'work', 'campus', 'Vidrohi', 'clinics', 'court', 'facility', 'case', 'emergency', 'time', 'patient', 'district', 'hospital', 'conditions', 'car', 'parking', 'court', 'premises', 'traffic', 'congestions', 'route', 'thousands', 'advocates', 'clients', 'court', 'day', 'emergency', 'proposal', 'state', 'govt', 'years', 'hospital', 'electro', 'cardiogram', 'facilities', 'floor', 'track', 'court', 'building', 'advocate', 'Kundan', 'Verma', 'Doctors', 'dispensaries', 'court', 'campus', 'vehicles', 'court', 'campus', 'manner', 'advocate', 'Deep', 'Narayan', 'Singh', 'ambulance', 'condition', 'driver', 'court', 'complex

In [94]:
# ranking the keywords by 5 most occuring keywords
from collections import Counter
for item in news_headlines:
  keyword_freq = Counter(item['keywords'])
  item['keywords'] = [word for word, freq in keyword_freq.most_common(5)]
  print(item['keywords'])

['court', 'Ranchi', 'emergency', 'advocate', 'campus']
['students', 'student', 'batch', 'ban', 'expulsion']
['hospital', 'baby', 'child', 'care', 'treatment']
['round', 'list', 'vacancy', 'delays', 'state']
['admissions', 'seats', 'round', 'students', 'India']
['seats', 'admissions', 'round', 'students', 'candidates']
['AFMS', 'health', 'Medical', 'services', 'challenges']
['Dr', 'S', 'accident', 'arm', 'holidays']
['pregnancy', 'court', 'board', 'MKCG', 'weeks']
['lecture', 'colleges', 'students', 'technology', 'holidays']
['court', 'Medical', 'LGBTQIA+', 'community', 'speakers']
['Aman', 'Bengaluru', 'drugs', 'Kerala', 'imaging']
['Rs', 'crore', '%', 'infrastructure', 'equipment']
['colleges', 'NMC', 'stipend', 'action', 'students']
['Nellie', 'medication', 'Beckey', 'boyfriend', 'dog']
['Kozhikode', 'Harshina', 'college', 'Dr', 'petition']
['medicine', 'Medical', 'eligibility', 'petitioners', 'certificates']
['Deepakraj', 'Trichy', 'college', 'year', 'car']
['email', 'SMS', 'police'

In [95]:
# Map keywords to topics
keyword_to_topic = {
    # Politics and Government
    "election": "Politics",
    "vote": "Politics",
    "government": "Politics",
    "parliament": "Politics",
    "prime minister": "Politics",
    "president": "Politics",
    "political party": "Politics",
    "election results": "Politics",
    "voting": "Politics",
    "democracy": "Politics",
    "cabinet": "Politics",
    "policy": "Politics",
    "law": "Politics",
    "bill": "Politics",
    "senate": "Politics",
    "congress": "Politics",
    "opposition": "Politics",
    "coalition": "Politics",
    "protest": "Politics",
    "strike": "Politics",
    "rally": "Politics",
    "campaign": "Politics",
    "candidate": "Politics",
    "election commission": "Politics",

    # Disasters and Accidents
    "flood": "Disasters",
    "earthquake": "Disasters",
    "cyclone": "Disasters",
    "tsunami": "Disasters",
    "landslide": "Disasters",
    "fire": "Disasters",
    "accident": "Disasters",
    "crash": "Disasters",
    "explosion": "Disasters",
    "storm": "Disasters",
    "hurricane": "Disasters",
    "tornado": "Disasters",
    "drought": "Disasters",
    "famine": "Disasters",
    "heatwave": "Disasters",
    "cold wave": "Disasters",
    "avalanche": "Disasters",
    "volcano": "Disasters",
    "rescue": "Disasters",
    "relief": "Disasters",
    "emergency": "Disasters",
    "evacuation": "Disasters",

    # Sports
    "cricket": "Sports",
    "football": "Sports",
    "soccer": "Sports",
    "tennis": "Sports",
    "olympics": "Sports",
    "world cup": "Sports",
    "match": "Sports",
    "tournament": "Sports",
    "championship": "Sports",
    "player": "Sports",
    "team": "Sports",
    "score": "Sports",
    "goal": "Sports",
    "stadium": "Sports",
    "coach": "Sports",
    "training": "Sports",
    "injury": "Sports",
    "victory": "Sports",
    "defeat": "Sports",
    "medal": "Sports",
    "athlete": "Sports",
    "sportsmanship": "Sports",

    # Technology
    "technology": "Technology",
    "innovation": "Technology",
    "smartphone": "Technology",
    "laptop": "Technology",
    "computer": "Technology",
    "internet": "Technology",
    "5G": "Technology",
    "AI": "Technology",
    "artificial intelligence": "Technology",
    "machine learning": "Technology",
    "robotics": "Technology",
    "cybersecurity": "Technology",
    "hacking": "Technology",
    "data breach": "Technology",
    "software": "Technology",
    "app": "Technology",
    "startup": "Technology",
    "gadget": "Technology",
    "drone": "Technology",
    "blockchain": "Technology",
    "cryptocurrency": "Technology",
    "bitcoin": "Technology",
    "ethereum": "Technology",
    "NFT": "Technology",
    "virtual reality": "Technology",
    "augmented reality": "Technology",

    # Entertainment
    "movie": "Entertainment",
    "film": "Entertainment",
    "actor": "Entertainment",
    "actress": "Entertainment",
    "director": "Entertainment",
    "bollywood": "Entertainment",
    "hollywood": "Entertainment",
    "music": "Entertainment",
    "song": "Entertainment",
    "album": "Entertainment",
    "concert": "Entertainment",
    "festival": "Entertainment",
    "award": "Entertainment",
    "oscar": "Entertainment",
    "grammy": "Entertainment",
    "television": "Entertainment",
    "TV show": "Entertainment",
    "series": "Entertainment",
    "celebrity": "Entertainment",
    "gossip": "Entertainment",
    "fashion": "Entertainment",
    "red carpet": "Entertainment",
    "premiere": "Entertainment",

    # Business and Economy
    "economy": "Business",
    "business": "Business",
    "market": "Business",
    "stock": "Business",
    "share": "Business",
    "investment": "Business",
    "trade": "Business",
    "export": "Business",
    "import": "Business",
    "GDP": "Business",
    "inflation": "Business",
    "unemployment": "Business",
    "bank": "Business",
    "finance": "Business",
    "tax": "Business",
    "budget": "Business",
    "recession": "Business",
    "startup": "Business",
    "entrepreneur": "Business",
    "profit": "Business",
    "loss": "Business",
    "merger": "Business",
    "acquisition": "Business",
    "IPO": "Business",

    # Health and Medicine
    "health": "Health",
    "medicine": "Health",
    "hospital": "Health",
    "doctor": "Health",
    "patient": "Health",
    "vaccine": "Health",
    "COVID": "Health",
    "pandemic": "Health",
    "epidemic": "Health",
    "disease": "Health",
    "virus": "Health",
    "infection": "Health",
    "treatment": "Health",
    "surgery": "Health",
    "mental health": "Health",
    "fitness": "Health",
    "nutrition": "Health",
    "diet": "Health",
    "exercise": "Health",
    "wellness": "Health",
    "pharmacy": "Health",
    "clinical trial": "Health",

    # Education
    "education": "Education",
    "school": "Education",
    "college": "Education",
    "university": "Education",
    "student": "Education",
    "teacher": "Education",
    "exam": "Education",
    "result": "Education",
    "admission": "Education",
    "scholarship": "Education",
    "degree": "Education",
    "course": "Education",
    "online learning": "Education",
    "literacy": "Education",
    "research": "Education",
    "thesis": "Education",
    "academic": "Education",
    "curriculum": "Education",
    "tuition": "Education",
    "campus": "Education",
    "alumni": "Education",

    # Environment
    "environment": "Environment",
    "climate change": "Environment",
    "global warming": "Environment",
    "pollution": "Environment",
    "deforestation": "Environment",
    "wildlife": "Environment",
    "conservation": "Environment",
    "recycling": "Environment",
    "sustainability": "Environment",
    "green energy": "Environment",
    "solar power": "Environment",
    "wind energy": "Environment",
    "carbon emissions": "Environment",
    "ecosystem": "Environment",
    "biodiversity": "Environment",
    "natural disaster": "Environment",
    "renewable energy": "Environment",
    "plastic waste": "Environment",
    "air quality": "Environment",
    "water conservation": "Environment",

    # Crime and Law
    "crime": "Crime",
    "murder": "Crime",
    "theft": "Crime",
    "robbery": "Crime",
    "fraud": "Crime",
    "scam": "Crime",
    "kidnapping": "Crime",
    "arrest": "Crime",
    "police": "Crime",
    "investigation": "Crime",
    "court": "Crime",
    "judge": "Crime",
    "verdict": "Crime",
    "sentence": "Crime",
    "prison": "Crime",
    "jail": "Crime",
    "bail": "Crime",
    "lawyer": "Crime",
    "trial": "Crime",
    "evidence": "Crime",
    "witness": "Crime",
    "gang": "Crime",
    "terrorism": "Crime",
    "cybercrime": "Crime",

    # Science and Research
    "science": "Science",
    "research": "Science",
    "discovery": "Science",
    "experiment": "Science",
    "space": "Science",
    "NASA": "Science",
    "astronomy": "Science",
    "physics": "Science",
    "chemistry": "Science",
    "biology": "Science",
    "genetics": "Science",
    "evolution": "Science",
    "innovation": "Science",
    "invention": "Science",
    "scientist": "Science",
    "lab": "Science",
    "theory": "Science",
    "hypothesis": "Science",
    "publication": "Science",
    "journal": "Science",
    "peer review": "Science",
    "climate science": "Science",
    "environmental science": "Science",

    # Miscellaneous
    "festival": "Miscellaneous",
    "celebration": "Miscellaneous",
    "event": "Miscellaneous",
    "holiday": "Miscellaneous",
    "travel": "Miscellaneous",
    "tourism": "Miscellaneous",
    "culture": "Miscellaneous",
    "tradition": "Miscellaneous",
    "religion": "Miscellaneous",
    "festival": "Miscellaneous",
    "art": "Miscellaneous",
    "literature": "Miscellaneous",
    "history": "Miscellaneous",
    "archaeology": "Miscellaneous",
    "museum": "Miscellaneous",
    "heritage": "Miscellaneous",
    "monument": "Miscellaneous",
    "festival": "Miscellaneous",
}

In [96]:
# Function to assign topics based on keywords
def assign_topic(keywords):
    for keyword in keywords:
        if keyword in keyword_to_topic:
            return keyword_to_topic[keyword]
    return "Other"

for item in news_headlines:
    item['topic'] = assign_topic(item['keywords'])

In [97]:
# count the topics
from collections import Counter
topic_count=Counter([item['topic'] for item in news_headlines])
topic_count

Counter({'Crime': 4,
         'Education': 5,
         'Health': 5,
         'Other': 13,
         'Disasters': 1,
         'Technology': 1,
         'Politics': 1})

In [98]:
import random

def generate_meta_description(title, keywords, max_length=160):
    """
    Generates a meta description using title and keywords.

    Parameters:
    - title (str): The headline of the news article.
    - keywords (list): A list of important keywords.
    - max_length (int): The maximum length of the meta description (default: 160).

    Returns:
    - str: A meta description.
    """


    base_description = f"{title}. This article covers topics related to "


    selected_keywords = ", ".join(random.sample(keywords, min(len(keywords), 5)))


    meta_description = base_description + selected_keywords + "."

    # Trim to max length if needed
    if len(meta_description) > max_length:
        meta_description = meta_description[:max_length-3] + "..."

    return meta_description

for item in news_headlines:
    item['meta_description'] = generate_meta_description(item['headline'], item['keywords'])


In [99]:
news_headlines

[{'headline': '‘Improvemedicalfacilities in court’',
  'link': 'https://timesofindia.indiatimes.com/city/ranchi/improve-medical-facilities-in-court/articleshow/118427773.cms',
  'published_date': 'Feb 20, 2025, 23:38 (IST)',
  'article_text': 'Ranchi: The \nRanchi District Bar Association\n (RDBA) general secretary Sanjay Virdrohi and the association members have demanded improved \nmedical facilities\n at the \nRanchi civil court\n in order to tackle emergency cases.\nThe demand was voiced following the demise of a Jharkhand high court advocate Suraj Kumar on Tuesday due to heart attack on the court premises and the death of a civil court lawyer S N Tiwari on Wednesday in a similar manner during legal work outside the campus.\nVidrohi said, "There are two clinics on the civil court premises but they are mostly meant for primary medical facility. In case of emergency crucial time would be lost to take the patient to the nearby district hospital due to chaotic conditions of car parking 

In [100]:
def generate_seo_title(headline, keywords):
    return f"{headline} - {', '.join(keywords)}"

for item in news_headlines:
    item['seo_title'] = generate_seo_title(item['headline'], item['keywords'])



# Storing the news data


In [108]:
import sqlite3


conn = sqlite3.connect('article.db')
cursor = conn.cursor()


cursor.execute('''
CREATE TABLE IF NOT EXISTS articles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    summary TEXT NOT NULL,
    seo_title TEXT NOT NULL,
    meta_description TEXT NOT NULL,
    publish_date DATE NOT NULL
)
''')


conn.commit()
conn.close()

In [72]:
# Example scraper logic
def scrape_medical_news():
    articles = []
    # Add your scraping logic here
    for news_item in news_headlines:
        article = pd.DataFrame({
            "title": news_item["headline"],  # Ensure this is unique
            "summary": news_item["summary"],
            "seo_title": news_item["seo_title"],
            "meta_description": news_item["meta_description"],
            "publish_date": news_item["publish_date"]
        })
        articles.append(article)
    return articles

scrape_medical_news

<function __main__.scrape_medical_news()>

In [73]:
articles

[(1,
  '‘Improvemedicalfacilities in court’',
  ' The demand was voiced following the demise of a Jharkhand high court advocate Suraj Kumar on Tuesday due to heart attack on the court premises and the death of a civil court lawyer S N Tiwari .',
  '‘Improvemedicalfacilities in court’ - court, Ranchi, emergency, advocate, campus',
  '‘Improvemedicalfacilities in court’. This article covers topics related to Ranchi, emergency, advocate, court, campus.',
  'Feb 20, 2025, 23:38 (IST)'),
 (2,
  '‘Improvemedicalfacilities in court’',
  ' Barmer Medical College authorities suspend six students accused of persistent sexual harassment . A sexual harassment committee meeting held Friday under chairperson Dr Kamla Verma . Parents of the offending students were called in person before college authorities .',
  '‘Improvemedicalfacilities in court’ - students, student, batch, ban, expulsion',
  '‘Improvemedicalfacilities in court’. This article covers topics related to expulsion, students, batch, ba

In [109]:
def save_article_to_db(article):
    conn = sqlite3.connect('article.db')
    cursor = conn.cursor()

    summary_text=article['summary'][0]['summary_text']
    cursor.execute('''
        INSERT INTO articles
        (title, summary, seo_title, meta_description, publish_date)
        VALUES (?, ?, ?, ?, ?)
    ''', (
        article['headline'],
        summary_text,
        article['seo_title'],
        article['meta_description'],
        article['published_date']
    ))

    conn.commit()
    conn.close()


for article in news_headlines:
    save_article_to_db(article)

In [110]:
def fetch_articles_from_db():
    conn = sqlite3.connect('article.db')
    cursor = conn.cursor()

    cursor.execute('SELECT * FROM articles ORDER BY publish_date DESC')
    articles = cursor.fetchall()

    conn.close()
    return articles

# Example usage
articles = fetch_articles_from_db()
for article in articles:
    print(article)

(15, "Man poisons his girlfriends' dog by mistake; know all about the shocking mix up", ' Beckey Carstens realized her boyfriend had accidentally given her dog, Nellie, human medication . Instead of the prescribed UTI medication, he mistakenly gave the dog Ambien, a sedative commonly used to treat insomnia in humans', "Man poisons his girlfriends' dog by mistake; know all about the shocking mix up - Nellie, medication, Beckey, boyfriend, dog", "Man poisons his girlfriends' dog by mistake; know all about the shocking mix up. This article covers topics related to Nellie, medication, boyfriend, Beckey,...", 'etimes.in /Feb 19, 2025, 22:30 (IST)')
(19, 'Hoax bomb threat to 2 Jaipur med colleges over Hindi language', ' A bomb threat email targeting two medical colleges in Jaipur, India, was sent two days ago . The threat was noticed by the SMS Medical College administration Saturday morning . Police declared the email a hoax after a thorough search by a team', 'Hoax bomb threat to 2 Jaipur 

In [121]:
import sqlite3
import json

# Connect to your SQLite database
conn = sqlite3.connect('article.db')
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

# Fetch all articles
cursor.execute('SELECT * FROM articles ORDER BY publish_date DESC')
articles = cursor.fetchall()

# Convert to JSON
with open('articles.json', 'w') as f:
    json.dump([dict(article) for article in articles], f)

conn.close()